In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.utils.vis_utils import model_to_dot
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K
import tensorflow as tf
from tqdm import tqdm_notebook

from data import MaskShifted2DSequence, MaskShiftedSingleImage2DSequence,MaskedUntouched2DSequence
from evaluate import psnr, ssim
from pdnet import pdnet, invnet
from pdnet_crop import pdnet_crop, invnet_crop

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
tf.logging.set_verbosity(tf.logging.INFO)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [5]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [6]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=8)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [7]:
run_params = {
#     'n_primal': 5,
#     'n_dual': 5,
#     'n_iter': 7,
#     'n_filters': 32,
    'n_primal': 2,
    'n_dual': 2,
    'n_iter': 2,
    'n_filters': 16,
}

n_epochs = 10
run_id = f'pdnet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'

In [8]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
# fix from https://github.com/bstriner/keras-tqdm/issues/31#issuecomment-516325065
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
# tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
# tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb.on_test_begin = lambda x,y:None
# tqdm_cb.on_test_end = lambda x,y:None

In [9]:
# model = pdnet(lr=1e-3, **run_params)
model = pdnet_crop(lr=1e-3, **run_params)
# simple_model = invnet(lr=1e-2, **run_params)
print(model.summary(line_length=150))

W0807 12:02:25.804550 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 12:02:25.817361 139772354914048 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:58: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0807 12:02:25.861593 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 12:02:25.870541 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.ra

Model: "model_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
kspace_input (InputLayer)                        (None, 640, None, 1)             0                                                                   
______________________________________________________________________________________________________________________________________________________
buffer_primal (Lambda)                           (None, 640, None, 2)             0                 kspace_input[0][0]                                
______________________________________________________________________________________________________________________________________________________
mask_input (InputLayer)                          (None, 640, None)           

In [10]:
model.fit_generator(
    train_gen, 
    steps_per_epoch=10, 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=10,
    verbose=0,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
    max_queue_size=100,
    use_multiprocessing=True,
    workers=35,
)

W0807 12:02:26.983860 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0807 12:02:27.277184 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1008: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0807 12:02:29.063830 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0807 12:02:29.065764 139772354914048 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Process ForkPoolWorker-46:
Process ForkPoolWorker-59:
Process ForkPoolWorker-42:
Process ForkPoolWorker-58:
Process ForkPoolWorker-69:
Process ForkPoolWorker-43:
Process ForkPoolWorker-24:
Process ForkPoolWorker-55:
Process ForkPoolWorker-50:
Process ForkPoolWorker-52:
Process ForkPoolWorker-7:
Process ForkPoolWorker-41:
Process ForkPoolWorker-65:
Process ForkPoolWorker-70:
Process ForkPoolWorker-63:
Process ForkPoolWorker-51:
Process ForkPoolWorker-67:
Process ForkPoolWorker-40:
Process ForkPoolWorker-33:
Process ForkPoolWorker-12:
Process ForkPoolWorker-47:
Process ForkPoolWorker-21:
Process ForkPoolWorker-8:
Process ForkPoolWorker-18:
Process ForkPoolWorker-14:
Process ForkPoolWorker-15:
Process ForkPoolWorker-35:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Process ForkPoolWorker-62:
Process ForkPoolWorker-22:
Process ForkPoolWorker-5:
Process ForkPoolWorker-9:
Process ForkPoolWorker-60:
Process ForkPoolWorker-16:
Process ForkPoolWorker-48:
Process ForkPoolWorker-17:
Process

  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/

Process ForkPoolWorker-29:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwar

KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in work

  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# # simple overfit trials

# data = train_gen[0]
# val_data = val_gen[0]
# model.fit(
#     x=data[0], 
#     y=data[1], 
#     validation_data=val_data, 
#     batch_size=data[0][0].shape[0], 
#     epochs=100,
#     verbose=2, 
#     shuffle=False,
# )

In [ ]:
# def crop_center_mc(img, cropx, cropy=None):
#     # taken from https://stackoverflow.com/questions/39382412/crop-center-portion-of-a-numpy-image/39382475
#     if cropy is None:
#         cropy = cropx
#     y, x = img.shape[1:]
#     startx = x//2 - (cropx//2)
#     starty = y//2 - (cropy//2)
#     return img[:, starty:starty+cropy, startx:startx+cropx]

# def from_pd_out_to_im_crop(imgs, crop):
#     res = np.fft.fftshift(imgs, axes=[1, 2])
#     res = crop_center_mc(res, 320)
#     return res

In [ ]:
# im_recos = model.predict_on_batch(data[0])
# im_simple = simple_model.predict_on_batch(data[0])

# import matplotlib.pyplot as plt
# import numpy as np
# focus_idx = 10
# fig, axs = plt.subplots(1, 3, sharex=True, sharey=True)
# axs[0].imshow(from_pd_out_to_im_crop(np.squeeze(data[1]), 320)[focus_idx])
# axs[1].imshow(from_pd_out_to_im_crop(np.squeeze(im_recos), 320)[focus_idx])
# axs[2].imshow(from_pd_out_to_im_crop(np.squeeze(im_simple), 320)[focus_idx])

In [ ]:
# gt = np.squeeze(data[1])
# pred = np.squeeze(im_recos)
# pred_simple = np.squeeze(im_simple)

# psnr(from_pd_out_to_im_crop(gt, 320), from_pd_out_to_im_crop(pred, 320)), psnr(from_pd_out_to_im_crop(gt, 320), from_pd_out_to_im_crop(pred_simple, 320))

In [ ]:
# ssim(from_pd_out_to_im_crop(gt, 320), from_pd_out_to_im_crop(pred, 320)), ssim(from_pd_out_to_im_crop(gt, 320), from_pd_out_to_im_crop(pred_simple, 320))

In [ ]:
# psnr(gt, pred), psnr(gt, pred_simple)

In [ ]:
# plt.figure()
# _ = plt.hist(
#     [
#         np.abs(np.squeeze(data[1][11])).flatten(),
#         np.abs(np.squeeze(im_recos[11])).flatten()
#     ], 
#     bins=100,
# )